## Alternus Vera - Final Notebook
### Functions which give accuracy of news for various factors and final polynomial equation
### Team Psychic-Pandas


In [ ]:
import pandas as pd
import numpy as np
import requests

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.model_selection import train_test_split
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.tokenize import RegexpTokenizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [26]:
train_df = pd.read_csv('train_final.csv')

In [28]:
train_df = train_df.drop(train_df[train_df.label == 'half-true'].index)
train_df = train_df.drop(train_df[train_df.label == 'mostly-true'].index)
train_df = train_df.drop(train_df[train_df.label == 'barely-true'].index)
train_df = train_df.drop(train_df[train_df.label == 'FALSE'].index)

In [29]:
train_df.loc[train_df['label'] == 'TRUE', 'label_value'] = 0 
train_df.loc[train_df['label'] == 'pants-fire', 'label_value'] = 1


In [30]:

test_data = pd.read_csv('/Users/andrew/Documents/MS - SJSU/Fall Sem 2018/CMPE 257 - Machine Learning/liar_dataset/test_data.csv')
test_data.columns = ["id", "label", "statement", "subject", "speaker", "speaker_title", "State", "party_affiliation", "barely_true", "false", "half_true", "mostly_true", "pants_on_fire","context"]


In [31]:
test_data = test_data.drop(test_data[test_data.label == 'half-true'].index)
test_data = test_data.drop(test_data[test_data.label == 'mostly-true'].index)
test_data = test_data.drop(test_data[test_data.label == 'barely-true'].index)
test_data = test_data.drop(test_data[test_data.label == 'FALSE'].index)

In [32]:
test_data.loc[test_data['label'] == 'TRUE', 'sensational_actual'] = 0 
test_data.loc[test_data['label'] == 'pants-fire', 'sensational_actual'] = 1 


In [33]:
test_data.loc[test_data['label'] == 'TRUE', 'label_value'] = 0
test_data.loc[test_data['label'] == 'pants-fire', 'label_value'] = 1 


In [34]:
statements = train_df['statement']
statements = list(statements)

In [37]:
sid = SentimentIntensityAnalyzer()
results = []
for sentence in statements:
    #print(sentence)
    sentiment_score = sid.polarity_scores(sentence)
    sentiment_score['statement'] = sentence
    results.append(sentiment_score)

In [38]:
sentiment_df = pd.DataFrame.from_records(results)
sentiment_df.head()

,compound,neg,neu,pos,statement
0,-0.5106,0.19,0.741,0.069,chicago bears starting quarterbacks last 10 ye...
1,0.0000,0.00,1.000,0.000,mccain opposed requirement government buy amer...
2,0.0000,0.00,1.000,0.000,says paul ryan still endorsing trump
3,0.0772,0.00,0.822,0.178,federal government thinks authority regulate t...
4,0.0000,0.00,1.000,0.000,austin city basically doubled size every 25 ye...


In [39]:
sentiment_df['variation']= abs(sentiment_df['pos'] - sentiment_df['neg'])

In [40]:
sentiment_df['label'] = 0
sentiment_df.loc[sentiment_df['compound'] > 0.2, 'label'] = 1
sentiment_df.loc[sentiment_df['compound'] < -0.2, 'label'] = -1
sentiment_df.head()

,compound,neg,neu,pos,statement,variation,label
0,-0.5106,0.19,0.741,0.069,chicago bears starting quarterbacks last 10 ye...,0.121,-1
1,0.0000,0.00,1.000,0.000,mccain opposed requirement government buy amer...,0.000,0
2,0.0000,0.00,1.000,0.000,says paul ryan still endorsing trump,0.000,0
3,0.0772,0.00,0.822,0.178,federal government thinks authority regulate t...,0.178,0
4,0.0000,0.00,1.000,0.000,austin city basically doubled size every 25 ye...,0.000,0


In [41]:
test_data[['sentiment_score','variation']] = sentiment_df[['label','variation']][['sentiment_score','variation']] = sentiment_df[['label','variation']]

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [42]:
test_data.head(5)

,id,label,statement,subject,speaker,speaker_title,State,party_affiliation,barely_true,false,half_true,mostly_true,pants_on_fire,context,sensational_actual,label_value,sentiment_score,variation
3,9524.json,pants-fire,When asked by a reporter whether hes at the ce...,"campaign-finance,legal-issues,campaign-adverti...",state-democratic-party-wisconsin,NaN,Wisconsin,democrat,5,7,2,2,7,a web video,1.0,1.0,0.0,0.178
4,5962.json,TRUE,Over the past five years the federal governmen...,"federal-budget,pensions,retirement",brendan-doherty,NaN,Rhode Island,republican,1,2,1,1,0,a campaign website,0.0,0.0,0.0,0.000
5,7070.json,TRUE,Says that Tennessee law requires that schools ...,"county-budget,county-government,education,taxes",stand-children-tennessee,Child and education advocacy organization.,Tennessee,none,0,0,0,0,0,in a post on Facebook.,0.0,0.0,1.0,0.107
7,12849.json,TRUE,Donald Trump is against marriage equality. He ...,"gays-and-lesbians,marriage",sean-patrick-maloney,Congressman for NY-18,New York,democrat,0,0,0,0,0,a speech at the Democratic National Convention,0.0,0.0,-1.0,0.333
11,11269.json,pants-fire,Denali is the Kenyan word for black power.,environment,viral-image,NaN,NaN,none,5,5,0,3,15,an image shared on Facebook,1.0,1.0,1.0,0.200


In [47]:
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(lowercase=True,stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
text_counts= cv.fit_transform(train_df['statement'])

In [48]:

train_df = train_df.replace([np.inf, -np.inf], np.nan).dropna(subset=["variation"], how="all")

In [49]:
test_data = test_data.replace([np.inf, -np.inf], np.nan).dropna(subset=["variation"], how="all")

## Calculating Weightages for  factors based on threshold value and Random Forest Classifier

* A Random Classifier was built to find the weightage of each factor.
* A for loop iterates between 0 and 1 to pick the best possible threshold that would give the maximum accuracy for each factor
* For each factor the model was trained with the factor score and label value in the dataset

### Sensationalism Weightage

In [162]:
max_accuracy_sensationalism = 0
max_accuracy_political_affiliation = 0
dict_threshold = {}
threshold_list = []
accuracy_list = []
X_train = train_df[['sensational_value','variation']]
y_train = train_df['label_value']
X_test = test_data[['sensational_actual','variation']]
y_test = test_data['label_value']
for thresholdsen in np.linspace(0,1,1000):
    clf=RandomForestClassifier(n_estimators=100)
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)


    accuracy = metrics.accuracy_score(y_test, y_pred)
    threshold_list.append(thresholdsen)
    accuracy_list.append(accuracy)
    if accuracy > max_accuracy_sensationalism:
        max_accuracy_sensationalism = accuracy
        max_threshold_sensationalism = thresholdsen
print("Max_accuracy",max_accuracy_sensationalism)
print("Max_threshold",max_threshold_sensationalism)
    

Max_accuracy 0.41935483870967744
Max_threshold 0.5825825825825826


### Political Affiliation Weightage

In [140]:
dataset_sample = pd.read_csv('test_with_finalscore.csv')
dataset_sample.head(5)

X = dataset_sample[['label','republican_score','democrat_score']]
X = X.loc[X['label'].isin(['true','pants-fire'])]
X.loc[X['label'] == 'true', 'label_value'] = 0 
X.loc[X['label'] == 'pants-fire', 'label_value'] = 1


cols_of_interest = ['republican_score','democrat_score']

X_doc = X[cols_of_interest]
y = X['label_value']


print(X_doc.shape)
print(y.shape)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_doc, y, random_state=7,test_size = 0.3)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics


max_accuracy_political_affiliation = 0
dict_threshold = {}
threshold_list = []
accuracy_list = []

for threshold in np.linspace(0,1,1000):
    clf=RandomForestClassifier(n_estimators=100)
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)


    accuracy = metrics.accuracy_score(y_test, y_pred)
    
    threshold_list.append(threshold)
    accuracy_list.append(accuracy)
    if accuracy > max_accuracy_political_affiliation:
        max_accuracy_political_affiliation = accuracy
        max_threshold_sensationalism = threshold
print("Max_accuracy",max_accuracy_political_affiliation)
print("Max_threshold",max_threshold_sensationalism)
    

(303, 2)
(303,)
(212, 2)
(91, 2)
(212,)
(91,)
Max_accuracy 0.6593406593406593
Max_threshold 0.22322322322322322


## Controversy Weightage

In [247]:
dataset_sample_contro = pd.read_csv('output_contro.csv')
dataset_sample_contro.head(5)


,Doc2vec_score,variance_level,label_id
0,0.004599,0.077,0
1,0.563970,0.098,1
2,0.662198,0.099,0
3,0.020447,0.394,0
4,0.604162,0.000,1


In [248]:

X = dataset_sample_contro[['label_id','Doc2vec_score','variance_level']]
#X = X.loc[X['label'].isin(['true','pants-fire'])]
#X.loc[X['label'] == 'true', 'label_value'] = 0 
#X.loc[X['label'] == 'pants-fire', 'label_value'] = 1


cols_of_interest = ['Doc2vec_score','variance_level']

X_doc = X[cols_of_interest]
y = X['label_id']


print(X_doc.shape)
print(y.shape)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_doc, y, random_state=7,test_size = 0.3)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics


max_accuracy_controversy = 0
dict_threshold = {}
threshold_list = []
accuracy_list = []

for threshold in np.linspace(0,1,1000):

    clf=RandomForestClassifier(n_estimators=100)

    clf.fit(X_train,y_train)

    y_pred=clf.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    threshold_list.append(threshold)
    accuracy_list.append(accuracy)
    if accuracy > max_accuracy_controversy:
        max_accuracy_controversy = accuracy
        max_threshold_controversy= threshold
print("Max_accuracy",max_accuracy_controversy)
print("Max_threshold",max_threshold_controversy)
    

(1000, 2)
(1000,)
(700, 2)
(300, 2)
(700,)
(300,)
Max_accuracy 0.5166666666666667
Max_threshold 0.2772772772772773


# Anjana Eldo - Sensationalism factor
### Function to determine sensationalism score

In [50]:
train_df_doc = pd.read_csv('train_final.csv')

* Used my sensationalism distionary to train the Doc2Vec model

In [51]:
sensational_words_vocab = ['assassin','attack','domest','secur','enforc','disast','nation','prepared','respons','recoveri','emerg','respond','militia','shoot','evacu','hostag','explos','crime','organ','gang','state','breach','threat','standoff','lockdown','bomb','riot','land','incid','suspici','nuclear','hazard','infect','outbreak','contamin','terror','epidem','critic','infrastructur','transport','grid','outag','disrupt','violenc','cartel','drug','narcot','shootout','traffic','kidnap','illeg','smuggl','qaeda','weapon','devic','improvis','bomber','suicid','hurrican','tornado','tsunami','earthquak','tremor','flood','storm','extrem','weather','forest','strand','wildfir','avalanch','blizzard','lighten','broadcast','cyber','ddo','denial','servic','malwar','phish','believ','support','isi','absolut','promis','societi','declar','islam','recess','price','market','stock','lotteri','sanctoin','sign','chang','climat','global','warm','kill','dead','horrifi','vicious','injur','beat','histori','polici','care','job','biographi','messag','feder','abort','energi','foreign','health','economi','educ','candid','ethic','elect','financi','bankruptci','corpor','technolog','divers','bipartisanship','consum','militari','debt','legal','homeland','deficit','iraq','patriot','campaign','water','immigr','correct','civil','crimin','hous','kagan','poverti','sexual','public','tax','poll','social','gun','incom','govern','congress','medicaid','famili','gay','disabl','debat','medicar','environ','stimulu','sport','agricultur','ebola','afghanistan','marriag','pension','children','religion','pundit','labor','veteran','israel','anim','autism','counti','welfar','trade','occupi','hunger','vote','union','bush','popul','scienc','alcohol','redistrict','transpar','marijuana','suprem','basebal','china','space','recreat','gambl','obama','food','censu','fake','congression','tourism','human','privaci','retir','women','sotomayor','urban','natur','worker','wealth','homeless','fire','death','budget','regul','busi','safeti','issu','financ','accomplish','right','justic','nomin','machin','amend','effici','wall','record','administr','advertis','spill','court','birth','cultur','news','rule','hampshir','penalti','updat','lesbian','street','certif','fact','week','colbert','report','tampa']


In [52]:
tagged_data = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(sensational_words_vocab)]

In [53]:
from gensim.models.doc2vec import Doc2Vec
max_epochs = 100
vec_size = 20
alpha = 0.025


d2v_model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
d2v_model.build_vocab(tagged_data)
#d2v_model.train(tagged_data, total_examples=d2v_model.corpus_count, epochs=10, start_alpha=0.002, end_alpha=-0.016)
for epoch in range(max_epochs):
    d2v_model.train(tagged_data,
                total_examples=d2v_model.corpus_count,
                epochs=d2v_model.iter)
    d2v_model.alpha -= 0.0002
    d2v_model.min_alpha = d2v_model.alpha
    
d2v_model.save("d2v.model.pa")
print("Model Saved")


/anaconda3/lib/python3.6/site-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


Model Saved


In [54]:
df_train_statements_d2v=train_df_doc[['statement','label']]
model= Doc2Vec.load("d2v.model")
texts=[]
for x in df_train_statements_d2v['statement']:
    texts.append(model.infer_vector(x))

##### Logistics Regression and Random Forrest for Doc2Vec

In [55]:
def encode_label(x):
    if (x == 'true' or x=='mostly-true' or x=='half-true'):
        return 1
    else:
        return 0
encode_label('true')

1

In [56]:

X=pd.DataFrame(texts)
y=df_train_statements_d2v['label'].map(lambda x:encode_label(x))
X_train,X_test,y_train,y_test=train_test_split(X, y,test_size = .3, random_state = 1)

In [57]:

from sklearn.linear_model import LogisticRegression
logisticRegr_doc2Vec = LogisticRegression(C=100)
logisticRegr_doc2Vec.fit(X_train, y_train)
pred = logisticRegr_doc2Vec.predict(X_test)
import pickle
s = pickle.dumps(logisticRegr_doc2Vec)

In [58]:
from sklearn import metrics
print(metrics.classification_report(y_test,pred))

             precision    recall  f1-score   support

          0       0.57      0.89      0.69       255
          1       0.45      0.12      0.19       195

avg / total       0.52      0.56      0.47       450



In [59]:
from sklearn.ensemble import RandomForestClassifier
rf_doc2vec = RandomForestClassifier(n_jobs=-1,n_estimators=50,max_depth=90)
rf_doc2vec.fit(X_train,y_train)
rf_pred = rf_doc2vec.predict(X_test)

In [60]:
from sklearn import metrics
print(metrics.classification_report(y_test,rf_pred))

             precision    recall  f1-score   support

          0       0.59      0.89      0.71       255
          1       0.55      0.18      0.27       195

avg / total       0.57      0.58      0.52       450



#### Detetcing sensationalism for test data

In [61]:

import pickle
def detect_sensationalism_calc(news):
    data_pred=[]
    data_pred.append(model.infer_vector(news))
    lrg_pa = pickle.loads(s)
    pred_conf=lrg_pa.predict_proba(data_pred)
    #print(pred_conf)
    return pred_conf[0][1]

In [62]:
class SensationalismPredictor:
    def __init__(self,news):
        self.news=news
    def predict(self):
        return detect_sensationalism_calc(self.news)

In [171]:
detect_sensationalism_calc("say anni list polit group support trimest abort demand")

0.46212868370061505

# Mudrita Chaturvedi - Political Affiliation and Political Bias Score Predictor 

In [65]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

### Load dataset

In [69]:
# Training data
dataset_train = pd.read_csv('train.tsv', delimiter = '\t', quoting = 3, header=None)
dataset_train.columns = ["id", "label", "statement", "subject", "speaker", "speaker_title", "State", "party_affiliation", "barely_true", "false", "half_true", "mostly_true", "pants_on_fire","context"]
dataset_train.head(5)

,id,label,statement,subject,speaker,speaker_title,State,party_affiliation,barely_true,false,half_true,mostly_true,pants_on_fire,context
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0,1,0,0,0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0,0,1,1,0,a floor speech.
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70,71,160,163,9,Denver
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7,19,3,5,44,a news release
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15,9,20,19,2,an interview on CNN


### Cleaning and preprocessing

In [70]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stop_words=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/andrew/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/andrew/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/andrew/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [71]:

from nltk.stem import PorterStemmer,WordNetLemmatizer
def text_preprocessing(df_base,column):
    df=df_base.copy()
    # lowercase the text
    df[column]=df[column].str.lower()
    # word tokenization
    df[column]=df[column].map(lambda x: nltk.word_tokenize(x))
    # remove stop words and non alphanumeric charaters
    df[column]=df[column].map(lambda x: [w for w in x if (not w in stop_words) and w.isalpha()])
    # lemmatization
    wordnet_lemmatizer = WordNetLemmatizer()
    df[column]=df[column].map(lambda x: [ wordnet_lemmatizer.lemmatize(w) for w in x])    
    # stemming
    porter = PorterStemmer()
    df[column]=df[column].map(lambda x: [porter.stem(w) for w in x] )
    return df

In [72]:
df_train=text_preprocessing(dataset_train,'statement')
df_train.head()

,id,label,statement,subject,speaker,speaker_title,State,party_affiliation,barely_true,false,half_true,mostly_true,pants_on_fire,context
0,2635.json,false,"[say, anni, list, polit, group, support, abort...",abortion,dwayne-bohac,State representative,Texas,republican,0,1,0,0,0,a mailer
1,10540.json,half-true,"[declin, coal, start, start, natur, ga, took, ...","energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0,0,1,1,0,a floor speech.
2,324.json,mostly-true,"[hillari, clinton, agre, john, mccain, vote, g...",foreign-policy,barack-obama,President,Illinois,democrat,70,71,160,163,9,Denver
3,1123.json,false,"[health, care, reform, legisl, like, mandat, f...",health-care,blog-posting,NaN,NaN,none,7,19,3,5,44,a news release
4,9028.json,half-true,"[econom, turnaround, start, end, term]","economy,jobs",charlie-crist,NaN,Florida,democrat,15,9,20,19,2,an interview on CNN


In [73]:
import gensim
from gensim.models.doc2vec import TaggedDocument
from gensim.models.doc2vec import Doc2Vec

### Creating dictionary

In [74]:
vocab_political_affiliation=['private accounts',
'trade agreement',
'American people',
'tax breaks',
'trade deficit',
'oil companies',
'credit card',
'nuclear option',
'war in Iraq',
'middle class',
'President budget',
'Republican party',
'change the rules',
'minimum wage',
'budget deficit',
'Republican senators',
'wildlife refuge',
'card companies',
'worker\'s rights',
'poor people',
'Republican leader',
'cut funding',
'American workers',
'living in poverty',
'Senate Republicans',
'fuel efficiency',
'national wildlife',
'veterans health care',
'congressional black caucus',
'billion in tax cuts',
'security trust fund',
'social security trust',
'privatize social security',
'American free trade',
'central American free',
'corporation for public broadcasting',
'additional tax cuts',
'pay for tax cuts',
'tax cuts for people',
'oil and gax companies',
'prescription drug bill',
'caliber sniper rifles',
'increase the minimum wage',
'system of checks and balances',
'middle class families',
'cut health care',
'civil rights movement',
'cuts to child support',
'drilling in the Arctic National',
'victims of gun violence',
'solvency of social security',
'Voting Rights Act',
'war in Iraq and Afghanistan',
'civil rights protections',
'credit card debt',
'Affordable Care Act',
'stem cell',
'natural gas',
'death tax',
'illegal aliens',
'class action',
'war on terror',
'embryonic cell',
'tax relief',
'illegal immigration',
'personal account',
'pass the bill',
'private property',
'border security',
'human life',
'human embryos',
'increase taxes',
'retirement accounts',
'government spending',
'national forest',
'minority leader',
'urge support',
'cell lines',
'cord blood',
'action lawsuits',
'economic growth',
'food program',
'hate crimes legislation',
'adult stem cells',
'oil for food',
'personal retirement accounts',
'energy and natural resources',
'hate crimes law',
'change hearts and minds',
'global war on terrorism',
'death tax repeal',
'housing and urban affairs',
'million jobs created',
'national flood insurance',
'private property rights',
'temporary worker program',
'class action reform',
'growth and job creation',
'reform social security',
'Obamacare',
'president',
'george',
'bush',
'administration',
'republican',
'democrats',
'barack',
'obama',
'hillary',
'clinton',
'donald',
'trump',
'senate',
'house'
]


In [75]:

tagged_doc_pa = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(vocab_political_affiliation)]

### Training and saving Model 

In [76]:
from gensim.models.doc2vec import Doc2Vec
max_epochs = 100
vec_size = 20
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_doc_pa)

for epoch in range(max_epochs):
    #print('iteration {0}'.format(epoch))
    model.train(tagged_doc_pa,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    
model.save("d2v.model_political.pa")
print("Model Saved")


Model Saved


In [77]:
df_train_statements_d2v=df_train[['statement','label']]
model= Doc2Vec.load("d2v.model_political.pa")
texts=[]
for x in df_train_statements_d2v['statement']:
    texts.append(model.infer_vector(x))


In [78]:
def replace_label(x):
    if (x == 'true' or x=='mostly-true' or x=='half-true'):
        return 1
    else:
        return 0
replace_label('true')

1

### Applying Logistic regression

In [79]:
X=pd.DataFrame(texts)
#y=df_train_statements_d2v[['label']]
y=df_train_statements_d2v['label'].map(lambda x:replace_label(x))
X_train,X_test,y_train,y_test=train_test_split(X, y,test_size = .3, random_state = 1)

In [80]:

from sklearn.linear_model import LogisticRegression
logisticRegr_D2V_PA = LogisticRegression(C=100)
logisticRegr_D2V_PA.fit(X_train, y_train)
lr_pred_pa = logisticRegr_D2V_PA.predict(X_test)
import pickle
s = pickle.dumps(logisticRegr_D2V_PA)

### Accuracy Score

In [81]:
from sklearn import metrics
print(metrics.classification_report(y_test,lr_pred_pa))

             precision    recall  f1-score   support

          0       0.25      0.00      0.00      1373
          1       0.55      1.00      0.71      1708

avg / total       0.42      0.55      0.40      3081



### Random Forest Classifier

In [82]:
from sklearn.ensemble import RandomForestClassifier
rf_D2V_PA = RandomForestClassifier(n_jobs=-1,n_estimators=50,max_depth=90)
rf_D2V_PA.fit(X_train,y_train)
rf_pred_pa = rf_D2V_PA.predict(X_test)

In [83]:
from sklearn import metrics
print(metrics.classification_report(y_test,rf_pred_pa))

             precision    recall  f1-score   support

          0       0.44      0.27      0.33      1373
          1       0.55      0.72      0.63      1708

avg / total       0.50      0.52      0.49      3081



In [84]:
import pickle
def political_affiliation_checker(news):
    data_pred=[]
    data_pred.append(model.infer_vector(news))
    lrg_pa = pickle.loads(s)
    pred_conf=lrg_pa.predict_proba(data_pred)
    #print(pred_conf)
    return pred_conf[0][1]

In [85]:
class PoliticalAffilicationDetector:
    def __init__(self,news):
        self.news=news
    def predict(self):
        return political_affiliation_checker(self.news)

### Score for mostly true data

In [86]:

political_affiliation_checker("Hillary Clinton agrees with John McCain by voting to give George Bush the benefit of the doubt on Iran.")

0.7534285074076726

### Score for false news

In [87]:
political_affiliation_checker("Health care reform legislation is likely to mandate free sex change surgeries.")

0.7331134246399988

## Political Bias

### Dictionary for democrat bias

In [88]:
democratic_phrases=['private accounts',
'trade agreement',
'American people',
'tax breaks',
'trade deficit',
'oil companies',
'credit card',
'nuclear option',
'war in Iraq',
'middle class',
'President budget',
'Republican party',
'change the rules',
'minimum wage',
'budget deficit',
'Republican senators',
'wildlife refuge',
'card companies',
'worker\'s rights',
'poor people',
'Republican leader',
'cut funding',
'American workers',
'living in poverty',
'Senate Republicans',
'fuel efficiency',
'national wildlife',
'veterans health care',
'congressional black caucus',
'billion in tax cuts',
'security trust fund',
'social security trust',
'privatize social security',
'American free trade',
'central American free',
'corporation for public broadcasting',
'additional tax cuts',
'pay for tax cuts',
'tax cuts for people',
'oil and gax companies',
'prescription drug bill',
'caliber sniper rifles',
'increase the minimum wage',
'system of checks and balances',
'middle class families',
'cut health care',
'civil rights movement',
'cuts to child support',
'drilling in the Arctic National',
'victims of gun violence',
'solvency of social security',
'Voting Rights Act',
'war in Iraq and Afghanistan',
'civil rights protections',
'credit card debt',
'Affordable Care Act']

### Dictionary for republican bias

In [89]:
republican_phrases=[
'stem cell',
'natural gas',
'death tax',
'illegal aliens',
'class action',
'war on terror',
'embryonic cell',
'tax relief',
'illegal immigration',
'personal account',
'pass the bill',
'private property',
'border security',
'human life',
'human embryos',
'increase taxes',
'retirement accounts',
'government spending',
'national forest',
'minority leader',
'urge support',
'cell lines',
'cord blood',
'action lawsuits',
'economic growth',
'food program',
'hate crimes legislation',
'adult stem cells',
'oil for food',
'personal retirement accounts',
'energy and natural resources',
'hate crimes law',
'change hearts and minds',
'global war on terrorism',
'death tax repeal',
'housing and urban affairs',
'million jobs created',
'national flood insurance',
'private property rights',
'temporary worker program',
'class action reform',
'growth and job creation',
'reform social security',
'Obamacare'
]

In [90]:
tagged_doc_republican = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(republican_phrases)]

tagged_doc_democrat = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(democratic_phrases)]

### Training and testing Model for political bias

In [91]:
from gensim.models.doc2vec import Doc2Vec
max_epochs = 100
vec_size = 20
alpha = 0.025

model_dem = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model_dem.build_vocab(tagged_doc_democrat)

for epoch in range(max_epochs):

    model_dem.train(tagged_doc_democrat,
                total_examples=model_dem.corpus_count,
                epochs=model_dem.iter)
    
    model_dem.alpha -= 0.0002
    
    model_dem.min_alpha = model_dem.alpha
    
model_dem.save("d2v.model_dem.pa")
print("Model Saved")

Model Saved


In [92]:
from gensim.models.doc2vec import Doc2Vec
max_epochs = 100
vec_size = 20
alpha = 0.025

model_rep = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model_rep.build_vocab(tagged_doc_republican)

for epoch in range(max_epochs):
    model_rep.train(tagged_doc_republican,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model_rep.alpha -= 0.0002
    # fix the learning rate, no decay
    model_rep.min_alpha = model.alpha
    
model.save("d2v.model_rep.pa")
print("Model Saved")

Model Saved


In [93]:
df_train_statements_d2v=df_train[['statement','label']]
model_dem= Doc2Vec.load("d2v.model_dem.pa")
texts=[]
for x in df_train_statements_d2v['statement']:
    texts.append(model_dem.infer_vector(x))

In [94]:
df_train_statements_d2v=df_train[['statement','label']]
model_rep= Doc2Vec.load("d2v.model_rep.pa")
texts=[]
for x in df_train_statements_d2v['statement']:
    texts.append(model.infer_vector(x))

In [95]:
def replace_label(x):
    if (x == 'true' or x=='mostly-true' or x=='half-true'):
        return 1
    else:
        return 0
replace_label('true')

1

In [96]:
X=pd.DataFrame(texts)
y=df_train_statements_d2v['label'].map(lambda x:replace_label(x))
X_train,X_test,y_train,y_test=train_test_split(X, y,test_size = .3, random_state = 1)

### Logistic regression for political bias

In [97]:
from sklearn.linear_model import LogisticRegression
logisticRegr_D2V_DEM = LogisticRegression(C=100)
logisticRegr_D2V_DEM.fit(X_train, y_train)
lr_pred_dem = logisticRegr_D2V_DEM.predict(X_test)
import pickle
s = pickle.dumps(logisticRegr_D2V_DEM)

In [98]:
from sklearn.linear_model import LogisticRegression
logisticRegr_D2V_REP = LogisticRegression(C=100)
logisticRegr_D2V_REP.fit(X_train, y_train)
lr_pred_rep = logisticRegr_D2V_REP.predict(X_test)
import pickle
s = pickle.dumps(logisticRegr_D2V_REP)

In [99]:
from sklearn import metrics
print(metrics.classification_report(y_test,lr_pred_dem))

             precision    recall  f1-score   support

          0       0.50      0.00      0.00      1373
          1       0.55      1.00      0.71      1708

avg / total       0.53      0.55      0.40      3081



In [100]:
from sklearn import metrics
print(metrics.classification_report(y_test,lr_pred_rep))

             precision    recall  f1-score   support

          0       0.50      0.00      0.00      1373
          1       0.55      1.00      0.71      1708

avg / total       0.53      0.55      0.40      3081



### Random forest for political bias

In [101]:
from sklearn.ensemble import RandomForestClassifier
rf_D2V_DEM = RandomForestClassifier(n_jobs=-1,n_estimators=50,max_depth=90)
rf_D2V_DEM.fit(X_train,y_train)
rf_pred_dem = rf_D2V_DEM.predict(X_test)

In [102]:
from sklearn import metrics
print(metrics.classification_report(y_test,rf_pred_dem))

             precision    recall  f1-score   support

          0       0.45      0.27      0.34      1373
          1       0.56      0.73      0.63      1708

avg / total       0.51      0.53      0.50      3081



In [103]:
import pickle
def political_affiliation_democrat(news):
    data_pred=[]
    data_pred.append(model.infer_vector(news))
    lrg_dem = pickle.loads(s)
    pred_conf=lrg_dem.predict_proba(data_pred)
    #print(pred_conf)
    return pred_conf[0][1]

In [104]:
class PoliticalBiasDemocrat:
    def __init__(self,news):
        self.news=news
    def predict(self):
        return political_affiliation_democrat(self.news)

In [105]:
from sklearn.ensemble import RandomForestClassifier
rf_D2V_REP = RandomForestClassifier(n_jobs=-1,n_estimators=50,max_depth=90)
rf_D2V_REP.fit(X_train,y_train)
rf_pred_rep = rf_D2V_REP.predict(X_test)

In [106]:
from sklearn import metrics
print(metrics.classification_report(y_test,rf_pred_rep))

             precision    recall  f1-score   support

          0       0.44      0.26      0.33      1373
          1       0.55      0.74      0.63      1708

avg / total       0.50      0.52      0.50      3081



In [107]:
import pickle
def political_affiliation_republic(news):
    data_pred=[]
    data_pred.append(model.infer_vector(news))
    lrg_pa = pickle.loads(s)
    pred_conf=lrg_pa.predict_proba(data_pred)
    #print(pred_conf)
    return pred_conf[0][1]

In [108]:
class PoliticalBiasRepublic:
    def __init__(self,news):
        self.news=news
    def predict(self):
        return political_affiliation_republic(self.news)

### Evaluating score for random news 

In [109]:
print(dataset_train['statement'][16])
print(dataset_train['label'][16],' ',dataset_train['party_affiliation'][16])
print('------------')
print(political_affiliation_checker(dataset_train['statement'][16]))
print(political_affiliation_republic(dataset_train['statement'][16]))
print(political_affiliation_democrat(dataset_train['statement'][16]))

"McCain opposed a requirement that the government buy American-made motorcycles. And he said all buy-American provisions were quote 'disgraceful.' "
true   democrat
------------
0.7900135943686248
0.6985716476754903
0.7475885967516733


In [110]:
print(dataset_train['statement'][12])
print(dataset_train['label'][12],' ',dataset_train['party_affiliation'][12])
print('------------')

print(political_affiliation_checker(dataset_train['statement'][12]))
print(political_affiliation_republic(dataset_train['statement'][12]))
print(political_affiliation_democrat(dataset_train['statement'][12]))

When Mitt Romney was governor of Massachusetts, we didnt just slow the rate of growth of our government, we actually cut it.
false   republican
------------
0.8456202713659947
0.8276168124941033
0.7533353080817765


# Vidya Nand Kumar Coimbatore - Controversy Predictor

In [112]:
vocab_doc2vec=['abus','account','administr','advantag','afghanistan','aid','america','american','amount','armi','attack','attorney','author','ban','bank','benefit','bill','border','budget','campaign','candid','cathol','chairman','charg','china','chines','church','citi','class','comment','compani','concern','congress','conserv','control','cost','countri','court','credit','crime','crimin','crisi','cut','debat','debt','defens','deficit','delay','democrat','diseas','dollar','drug','economi','educ','effect','egypt','elect','enforc','expect','famili','februari','fight','financ','fiscal','forc','fund','ga','germani','goal','govern','gun','health','hous','immigr','inaccuraci','india','inform','insur','invest','investig','iran','israel','job','judg','justic','kill','korea','labor','land','law','lawmak','lawsuit','leadership','legisl','market','marriag','media','mexico','militari','money','murder','nation','news','number','obama','offens','offici','oil','oklahoma','parent','parti','patient','peac','polic','polici','polit','poll','popul','power','presid','price','primari','prison','progress','project','race','rais','rate','reason','reform','republican','restrict','rule','russia','russian','sale','school','sector','secur','senat','sex','shoot','shot','societi','sourc','spend','statu','stock','store','strategi','strike','support','syria','syrian','tax','threat','trial','unemploy','union','usa','victim','violenc','vote','voter','war','washington','weapon','whether','world','worth']

In [113]:
tagged_doc_2vec = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(vocab_doc2vec)]

In [114]:
from gensim.models.doc2vec import Doc2Vec
max_epochs = 100
vec_size = 20
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_doc_2vec)

for epoch in range(max_epochs):
   
    model.train(tagged_doc_2vec,
                total_examples=model.corpus_count,
                epochs=model.iter)
   
    model.alpha -= 0.0002
    
    model.min_alpha = model.alpha
    
model.save("doc2vec_model")
print("Model Saved")

Model Saved


In [115]:
df_train_statements_d2v=train_df_doc[['statement','label']]
model_dem= Doc2Vec.load("doc2vec_model")
texts=[]
for x in df_train_statements_d2v['statement']:
    texts.append(model_dem.infer_vector(x))

In [116]:
def replace_label(x):
    if (x == 'true' or x=='mostly-true' or x=='half-true'):
        return 1
    else:
        return 0
replace_label('true')

1

In [117]:
X=pd.DataFrame(texts)
y=df_train_statements_d2v['label'].map(lambda x:replace_label(x))
X_train,X_test,y_train,y_test=train_test_split(X, y,test_size = .3, random_state = 1)

In [118]:

from sklearn.linear_model import LogisticRegression
logisticRegr_doc2Vec = LogisticRegression(C=100)
logisticRegr_doc2Vec.fit(X_train, y_train)
pred = logisticRegr_doc2Vec.predict(X_test)
import pickle
s = pickle.dumps(logisticRegr_doc2Vec)

In [119]:
from sklearn import metrics
print(metrics.classification_report(y_test,pred))

             precision    recall  f1-score   support

          0       0.57      0.91      0.70       255
          1       0.46      0.10      0.16       195

avg / total       0.52      0.56      0.47       450



In [120]:
from sklearn.ensemble import RandomForestClassifier
rf_doc2vec = RandomForestClassifier(n_jobs=-1,n_estimators=50,max_depth=90)
rf_doc2vec.fit(X_train,y_train)
rf_pred = rf_doc2vec.predict(X_test)

In [121]:
from sklearn import metrics
print(metrics.classification_report(y_test,rf_pred))

             precision    recall  f1-score   support

          0       0.56      0.86      0.68       255
          1       0.42      0.13      0.20       195

avg / total       0.50      0.54      0.47       450



In [122]:
import pickle
def controversy_checker(news):
    data_pred=[]
    data_pred.append(model.infer_vector(news))
    lrg_pa = pickle.loads(s)
    pred_conf=lrg_pa.predict_proba(data_pred)
    #print(pred_conf)
    return pred_conf[0][1]

In [123]:
class ControversyDetector:
    def __init__(self,news):
        self.news=news
    def predict(self):
        return controversy_checker(self.news)

In [124]:
controversy_checker("say anni list polit group support trimest abort demand")

0.44763847494470127

# Anjana Kamath Miyar - Topic features 

In [225]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

countV = CountVectorizer()
train_count = countV.fit_transform(train_df['statement'].values)
y_train = train_df['label']

test_count =  countV.transform(test_data['statement'].values)
y_test = test_data['label']

tfidfV = TfidfVectorizer()
train_tfidf = tfidfV.fit_transform(train_df['statement'].values)
test_tfidf =  tfidfV.transform(test_data['statement'].values)


### Multinomial Naive Bayes

In [226]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix

clf = MultinomialNB() 
clf.fit(train_tfidf, y_train)
pred = clf.predict(test_tfidf)
score = accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score)

accuracy:   0.645


### Logistic Regression

In [227]:
log_clf = LogisticRegression() 
log_clf.fit(train_tfidf, y_train)
pred = log_clf.predict(test_tfidf)
score = accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score)

accuracy:   0.634


### SVM

In [229]:
from sklearn import svm
svm_clf = svm.LinearSVC() 
svm_clf.fit(train_tfidf, y_train)
pred = svm_clf.predict(test_tfidf)
score = accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score)

accuracy:   0.613


### PassiveAggressiveClassifier

In [230]:
from sklearn.linear_model import PassiveAggressiveClassifier

pa_clf = PassiveAggressiveClassifier(n_iter=50)
pa_clf.fit(train_tfidf, y_train)
pred = pa_clf.predict(test_tfidf)
score = accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score)

accuracy:   0.624


### RandomForestClassifier

In [231]:
rfc_clf = RandomForestClassifier()
rfc_clf.fit(train_tfidf, y_train)
pred = rfc_clf.predict(test_tfidf)
score = accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score)

accuracy:   0.613


In [331]:
def features_detection(var):    
#retrieving the best model for prediction call
    load_model = pickle.load(open('final_model.sav', 'rb'))
    prediction = load_model.predict([var])
    prob = load_model.predict_proba([var])
    #print("The given statement is ",prediction[0])
    #print("The truth probability score is ",prob[0][1])
    return prob[0][1]

In [317]:
input_news = 'Building a wall on the U.S.-Mexico border will take literally years'
detecting_fake_news(input_news)

The given statement is  True
The truth probability score is  0.568017336920146


#### Based on the maximum accuracy calculated by running the above algorithms I chose topic features factor to be of 0.64 accuracy

## Final Polynomial Equation

In [373]:
def fakedetection(news_article):
    
    sensationalism_factor= detect_sensationalism_calc(news_article)
    
    political_affiliation_republic_factor= political_affiliation_republic(news_article)
    political_affiliation_democrat_factor = political_affiliation_democrat(news_article)
    political_affiliation_checker_factor = political_affiliation_checker(news_article)

    controversy_checker_factor = controversy_checker(news_article)
    
    topic_features_factor = features_detection(news_article)
    
    sensationalism_weight=.26
    political_affilication_republic_weight= 0.3
    political_affilication_democrat_weight = 0.4
    controversy_checker_weight=.25
    topic_features_factor_weight = 0.40
    
    print('sensationalism_factor:',sensationalism_factor)
    print('political_bias_republic_factor:',political_affiliation_republic_factor)
    print('political_bias_democrat_factor:',political_affiliation_democrat_factor)
    print('political_affiliation_checker_factor:',political_affiliation_checker_factor)
    print('controversy_checker_factor:',controversy_checker_factor)
    print('topic_features_factor:',topic_features_factor)
    total_perc_weight = sensationalism_weight+political_affilication_republic_weight+political_affilication_democrat_weight+political_affilication_checker_weight+ controversy_checker_weight+topic_features_factor_weight
    
    return ((sensationalism_factor*sensationalism_weight)+
            (political_affiliation_republic_factor*political_affilication_republic_weight)+
            (political_affiliation_democrat_factor*political_affilication_democrat_weight)+
            (political_affiliation_checker_factor*political_affilication_checker_weight)+
            (controversy_checker_factor*controversy_checker_weight))

## Evaluating True News
* We took false news from test data and checked with our fake news detection function. The values we got were close to 0 and it showed that the news is closer to being true.That is false probability

In [375]:
input_news = 'Each year, 18,000 people die in America because they dont have health care'
print('\033[1m' +'probability of news to be false '+'\033[1m',fakedetection(input_news))

sensationalism_factor: 0.32954110681275695
political_bias_republic_factor: 0.2657501496052827
political_bias_democrat_factor: 0.31977115047872084
political_affiliation_checker_factor: 0.3579305842509412
controversy_checker_factor: 0.32344675267465894
topic_features_factor: 0.5366298204400898
probability of news to be false  0.3741758810130547


In [376]:
input_news = 'Building a wall on the U.S.-Mexico border will take literally years'
print('\033[1m' +'probability of news to be false '+'\033[1m',fakedetection(input_news))

sensationalism_factor: 0.44787472588807736
political_bias_republic_factor: 0.3606545126394851
political_bias_democrat_factor: 0.40699627765655344
political_affiliation_checker_factor: 0.4006949085818859
controversy_checker_factor: 0.4154589177345162
topic_features_factor: 0.568017336920146
probability of news to be false  0.4913070230189961


## Evaluating False News 
* We took false news from test data and checked with our fake news detection function. The values we got were close to one and it gave the percentage of falsity in the news.

In [363]:
input_news = 'Says John McCain has done nothing to help the vets'
print('\033[1m' +'probability of news to be false '+'\033[1m',fakedetection(input_news))

sensationalism_factor: 0.37573535788536233
political_bias_republic_factor: 0.37843643548350486
political_bias_democrat_factor: 0.3424823517760719
political_affiliation_checker_factor: 0.39406114477252097
controversy_checker_factor: 0.3598124218843832
topic_features_factor: 0.5835102088200236
probability of news to be false  0.8500470353171607


In [364]:
input_news = 'Wisconsin is on pace to double the number of layoffs this year'
print('\033[1m' +'probability of news to be false '+'\033[1m',fakedetection(input_news))

sensationalism_factor: 0.32635613699711863
political_bias_republic_factor: 0.34332511439263347
political_bias_democrat_factor: 0.3788256707271877
political_affiliation_checker_factor: 0.3115139203438204
controversy_checker_factor: 0.38590214228785774
topic_features_factor: 0.5957055348392479
probability of news to be false  0.8071086526720894


# Conclusion

* Each individual factor was given a weight depending on the factor score we got by carrying out various algorithms.
* Sensationalism score was computed by applying LDA using TF-IDF, BOW, Word2Vec sentiment analysis ,cosine similarity and Doc2Vec. Trained a model using the sensationalism score and variation and got a weightage of 0.58 which gave maximum accuracy of 0.41 using Random Forest Classifier.
* Political Affiliation gave republican and democratic score based on the algorithms performed on it - sentiment analysis, Doc2Vec, BOW, LDA and cosine similarity. Trained the model using republican and democratic score and got a weightage of 0.22 with an accuracy of 0.65 using Random Forest Classifier.
* Controversy score was computed by applying Doc2Vec, LDA, BOW and cosine similarity. The model was trained using Random Forest Classifier, which provided threshold as 0.27 and maximum accuracy as 0.51.

* After getting the weightages for each factor we applied doc2vec to determine the truth or false depending on each factor.
* Percentages of each factor gave us the percentage of falsity and truth in the news.

